In [ ]:
import sys
import mkl
mkl.get_max_threads()

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
#from mpl_toolkits.mplot3d import Axes3D
from sklearn.svm import LinearSVC 
from sklearn.preprocessing import StandardScaler
from scipy.ndimage.measurements import label
from scipy.spatial.distance import cdist

from skimage.feature import hog
import cv2
from glob import glob
import time
import os
import pickle

import pickle
from sklearn.model_selection import train_test_split
from keras.preprocessing import image
from keras.models import load_model, Model, Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Dropout, Lambda, Input, Concatenate, Add, Cropping2D
from keras.layers.convolutional import Conv2D
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical

import pipeline as pl
import VehicleDetector as vd


# Reload custum module everytime
%reload_ext autoreload
%aimport pipeline, VehicleDetector
%autoreload 1

## Defining the Scanning Windows

In [ ]:
kw_windows_far = {'center_coords':(900, 600),
               'xy_area_size':(800, 220),
               'window_widths':180,
               'width_height_ratio': 0.75,
               'overlap':0.1}
kw_windows_mid = {'center_coords':(900,550),
               'xy_area_size':(800, 150),
               'window_widths':140,
               'width_height_ratio': 0.75,
               'overlap':0.1}
kw_windows_near = {'center_coords':(900,450),
               'xy_area_size':(800, 150),
               'window_widths':120,
               'width_height_ratio': 0.75,
               'overlap':0.1}
kw_windows_right = {'center_coords':(1100,500),
               'xy_area_size':(400, 400),
               'window_widths':280,
               'width_height_ratio': 1,
               'overlap':0.1}

windows_far   = pl.full_scan_windows(**kw_windows_far)
windows_mid   = pl.full_scan_windows(**kw_windows_mid)
windows_near  = pl.full_scan_windows(**kw_windows_near)
windows_right = pl.full_scan_windows(**kw_windows_right)

windows = []
for w in windows_far:
    windows.append(w)
for w in windows_mid:
    windows.append(w)
for w in windows_near:
    windows.append(w)
for w in windows_right:
    windows.append(w)

## Loading the Model

In [ ]:
scalers = pickle.load( open('./scalers.p', 'rb'))   
model = load_model('./model.h5')
model.summary()

## Detect Vehicles and Save Project Video

In [ ]:
video_filename = './project_video.mp4'
n = 905
dn = 15

objects  = vd.Detector(model)
vehicles = vd.Detector(model)
distances = []

def create_project_video():
    
    MAX_LAYERS = 6
    MAX_LOCAL_LAYERS = 2
    MIN_DISTANCE = 100
    timer = 0
    def detect_vehicles(video_frame):
        nonlocal timer
        img = (video_frame/255.0).astype(np.float32)
        window_img = img.copy()
            
        if timer > 30:
            # ++++++++++++++++++++++ F U L L  S C A N ++++++++++++++++++++++
            timer = 0
            objects.detections = []
            boxes, predictions = vehicles.full_scan(img, windows, MAX_LAYERS, verbose=0)
            for box, prediction in zip(boxes, predictions):
                objects.detections.append(vd.Car(img, box, prediction, scalers))
            for car in vehicles.detections:
                for box, prediction, detection in zip(boxes, predictions, objects.detections):
                    p1 = np.array(car.center_coords).reshape(1,-1)
                    p2 = np.array(detection.center_coords).reshape(1,-1)
                    dist = cdist(p1, p2) 
                    distances.append(dist)
                    if dist < MIN_DISTANCE:
                        objects.detections.remove(detection)
                        boxes.remove(box)
                        predictions.remove(prediction)
            for box, prediction in zip(boxes, predictions):
                vehicles.detections.append(vd.Car(img, box, prediction, scalers))

        
        # ++++++++++++++++++++++ L O C A L  S C A N ++++++++++++++++++++++
        timer += 1
        local_boxes, predictions = vehicles.local_scan(img, MAX_LOCAL_LAYERS, verbose=0)
        window_img = pl.draw_boxes(window_img, local_boxes, color=(1, 0, 0), thick=2) 
            
        result = (window_img*255).astype(np.uint8)
        return result

    video = VideoFileClip(video_filename)
    detection_video = video.fl_image( detect_vehicles )
    %time detection_video.write_videofile("./vehicle_detection_video.mp4", audio=False)
    return detection_video
detection_video = create_project_video()
 